In [228]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [229]:
import json
f = open("/content/drive/MyDrive/Finance QA/data/valid_unfiltered_qa_pairs_with_data.json",)
data = json.load(f)

print(len(data))
f.close()

5296


List of (title, context-segment, question, answer)

In [230]:
from pprint import pprint

In [231]:
# !pip install -U spacy 
!python -m spacy download en_core_web_sm

     |████████████████████████████████| 13.6 MB 76 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [232]:
import spacy

nlp = spacy.load('en_core_web_sm')

sentence = data[20]['Q_A_pairs'][3][1]

doc = nlp(sentence)

print(sentence)

for ent in doc.ents:

  print(ent.text, ent.start_char, ent.end_char, ent.label_)


Okay, Tom. Thanks for the question. I'll take the first part of it, then I'll hand it over to Richard. And I do agree with the way you position the question. And I think the longer we wait to take on some of these challenges, which are also opportunities, the more defensive position we would be in. We made a conscious decision that we needed to take an offensive position, and we needed to readjust our strategic initiatives, and we needed to ramp up our investments and we needed to – I'd call it, supercharge the level of change that we are doing in our company. We have this proud company that's been very successful, and we're all glad to be a part of that.
But we issued a challenge to our people, and that was that if we continue to do business the same way we do today, and this was a year or two ago, that we didn't see as rosy a future for the company or for the shareholders as we would if we took a much more aggressive stand that if we charged into the future and that's what you've bee

In [233]:
import spacy

nlp = spacy.load('en_core_web_sm')

def jaccard_distance(list1, list2):
    intersection = len(list(set(list1).intersection(list2)))
    union = (len(list1) + len(list2)) - intersection
    return float(intersection) / union

def get_ner_set(sentence):
    ret = set()

    doc = nlp(sentence)

    for ent in doc.ents:
      if ent.label_ == 'PERCENT' or  ent.label_ == 'MONEY':
        ret.add(ent.text)

    return ret

In [240]:
# Loop through each document
import spacy

nlp = spacy.load('en_core_web_sm')

dataset = list()

for document in data:
  title = document['title']
  executives = document['executives']
  expo = document['expo']
  Q_A_pairs = document['Q_A_pairs']

  expo_sets = list()

  for i in range(len(expo)):
    expo_set = get_ner_set(expo[i])
    expo_sets.append(expo_set)

  for q_a in Q_A_pairs:
    q, a = q_a

    q_set = get_ner_set(q)
    a_set = get_ner_set(a)

    combined = q_set.union(a_set)

    best_score = -1
    best_expo = -1

    for i, expo_set in enumerate(expo_sets):
      if len(combined) == 0 or len(expo_set) == 0:
        continue
      score = jaccard_distance(combined, expo_set)

      if score > best_score:
        best_score = score
        best_expo = expo[i]

    if best_score > 0.7:
      dataset.append((title, best_score, best_expo, q, a))


In [234]:
# def get_context_around(expo, i):
#   if i == 0:
#     return expo[0] + "\n" + expo[1]
#   elif i == len(expo) -1:
#     return expo[len(expo) -2] + "\n" + expo[len(expo) -1]
#   else:
#     return expo[i-1] + "\n" + expo[i] + "\n" + expo[i+1]

# # Loop through each document
# import spacy

# nlp = spacy.load('en_core_web_sm')

# dataset = list()

# for document in data[:100]:
#   title = document['title']
#   executives = document['executives']
#   expo = document['expo']
#   Q_A_pairs = document['Q_A_pairs']

#   expo_sets = list()

#   for i in range(len(expo)):
#     context = get_context_around(expo, i)
#     expo_set = get_ner_set(context)
#     expo_sets.append(expo_set)

#   for q_a in Q_A_pairs:
#     q, a = q_a

#     q_set = get_ner_set(q)
#     a_set = get_ner_set(a)

#     combined = q_set.union(a_set)

#     best_score = -1
#     best_expo = -1
#     best_i = -1

#     for i, expo_set in enumerate(expo_sets):
#       if len(combined) == 0 or len(expo_set) == 0:
#         continue
#       score = jaccard_distance(combined, expo_set)

#       if score > best_score:
#         best_score = score
#         best_expo = expo[i]
#         best_i = i

#     if best_score > 0.7:
#       dataset.append((title, best_score, get_context_around(expo, best_i), q, a))


KeyboardInterrupt: ignored

In [241]:
print(len(dataset))

1174


In [242]:
import json
with open('/content/drive/MyDrive/Finance QA/data/smaller_context_question_answer_pairs_full.json', 'w') as f: 
    json.dump(dataset, f)
    f.close()


In [243]:
import json
with open('/content/drive/MyDrive/Finance QA/data/smaller_context_question_answer_pairs_full.json') as f: 
    dataset = json.load(f)
    f.close()

In [247]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [254]:
expo_lens = list()
question_lens = list()
answer_lens = list()

for pair in dataset:
  _, _, expo, question, answer = pair

  from nltk.tokenize import word_tokenize

  expo_lens.append(len(expo.split('\n')))
  question_lens.append(len(question.split('\n')))
  answer_lens.append(len(answer.split('\n')))


In [257]:
import numpy as np
  
r1 = np.mean(expo_lens)
print("\nMean: ", r1)
  
r2 = np.std(expo_lens)
print("\nstd: ", r2)


Mean:  1.0

std:  0.0


In [239]:
i = 4
title, best_score, best_expo, q, a = dataset[i]

In [11]:
pprint(title)
pprint(best_score)
pprint(best_expo)
pprint(q)
pprint(a)

['United Rentals, Inc. (NYSE:URI) Q1 2017 Earnings Conference Call April 20, '
 '2017 11:00 AM ET']
1.0
("So here we are in 2017 and I'm pleased that we reported such a strong start "
 'to the year. In January, I described 2017 as a critical period in terms of '
 'both positioning ourselves with our broader customer base and driving new '
 "efficiencies in our operations, and we're executing well on both fronts.\n"
 'Many of the actions we took in the quarter were designed to strengthen our '
 'earning power in future periods. This includes finalizing the NES '
 "transaction, which we closed as planned on April 3. Internally, I'm "
 'impressed how quickly our combined workforce has adopted the mindset of a '
 'single team, and customer reaction to the acquisition has been positive.\n'
 'So I want to take this opportunity to publicly welcome more than a thousand '
 'new employees to United Rentals family. They are great addition to the team. '
 'We also got off to a solid start on Proje

In [195]:
pprint(data[0].keys())

dict_keys(['title', 'executives', 'analysts', 'expo', 'Q_A_pairs'])


In [7]:
pprint(data[0]['expo'])

['United Continental Holdings, Inc. (NASDAQ:UAL) Q1 2017 Results Earnings '
 'Conference Call April 18, 2017 10:30 AM ET',
 'Executives',
 'Julie Yates Stewart - Managing Director, IR',
 'Oscar Munoz - CEO',
 'Scott Kirby - President',
 'Andrew Levy - EVP and CFO',
 'Gregory Hart - EVP and COO',
 'Analysts',
 'Helane Becker - Cowen and Company',
 'Kevin Crissey - Citigroup',
 'Michael Linenberg - Deutsche Bank',
 'Hunter Keay - Wolfe Research',
 'Savanthi Syth - Raymond James',
 'Darryl Genovesi - UBS',
 'Jamie Baker - JPMorgan',
 'Joseph DeNardi - Stifel & Nicolaus & Co.',
 'Brandon Oglenski - Barclays Capital',
 'Andrew George Didora - Bank of America Merrill Lynch',
 'Rajeev Lalwani - Morgan Stanley',
 'Duane Pfennigwerth - Evercore ISI',
 'David Koenig - Associated Press',
 'Michael Sasso - Bloomberg News',
 'Ted Reed - THE STREET',
 'Edward Russell - Flightglobal',
 'Operator',
 'Good morning and welcome to United Continental Holdings Earnings Conference '
 "Call for the First Qua

In [12]:
pprint(data[0]['Q_A_pairs'][1][0])

("No. I was just actually -- I mean, I don't want to beat this up. I just was "
 'going to change the subject and actually ask you about regional -- regions '
 'in the domestic market. I know you were talking about improvement in various '
 'domestic markets, and I wonder if there are any specific areas of '
 'improvement that you would call out as being noteworthy. And that was all.')


In [ ]:
pprint(data[0]['Q_A_pairs'][0][1])

("We had a lot of communication with our corporate accounts, and we've had "
 'appropriate questions and concerns, particularly with regard to our '
 "corporate accounts, with whom we have a good relationship and we've already "
 "communicated with. We feel pretty good about the communications that we've "
 'had so far and in our ability to reassure them and explain things like '
 'overbooking.\n'
 'Some of the steps that we will do with the broader public on our April 30 '
 "launch but some of that with our corporate accounts, we've been able to "
 'accelerate some of that -- some of their specific concerns. And our sales '
 'team has done a wonderful job of being out, talking to them if there has '
 'been a concern from corporate accounts, which has been totally appropriate.\n'
 "We feel like we've managed that pretty well. And our corporate accounts are "
 'largely supportive. They want us to fix this. They want us to do the right '
 'thing. But they believe in us and believe that w